In [15]:
        # Rotate Green Line
# # David H Nguyen, PhD. 

# This version should print the name of the file that caused an error, so you know which file it was. 

### Instructions ###
# 1. Input files MUST be in PAIRS. 1) The original image, 2) a copy of the original image. 
#   The copy image must have the characters " copy" at the end of its file name.
# 2. The copy image must have a bright green line in it. The original image cannot have green color in it. 
# 3. The rotated images will appear in the same directory that contains the input images. 
#    Rotated images will have the suffix "rotated" at the end of their file name.


In [1]:
import cv2
import numpy as np
import os


In [59]:
# Step 1 - Set Working Directory 
os.chdir('/Users/davidnguyen/Downloads/AML FAB subtypes_at-scale/AML M6/AML M6 both/AML M6 both_masks_crop_thicken_greenline')
path = os.getcwd()
contents = os.listdir(path)
hidden_files = []

# Remove hidden files (Mac-related)
for item in contents:
    if item.startswith("."):
        hidden_files.append(item)

for item in hidden_files:
    contents.remove(item)

contents

['AML-Raul_M6cn_1_both_plus100margin_thickened.tif',
 'AML-Raul_M6cn_9_both_plus100margin_thickened copy.tif',
 'AML-Raul_M6cn_8_both_plus100margin_thickened copy.tif',
 'AML-Raul_M6cn_5_both_plus100margin_thickened.tif',
 'AML-Raul_M6cn_13_both_plus100margin_thickened.tif',
 'AML-Raul_M6cn_9_both_plus100margin_thickened.tif',
 'AML-Raul_M6cn_2_both_plus100margin_thickened.tif',
 'AML-Raul_M6cn_6_both_plus100margin_thickened.tif',
 'AML-Raul_M6cn_10_both_plus100margin_thickened.tif',
 'AML-Raul_M6cn_8_both_plus100margin_thickened.tif',
 'AML-Raul_M6cn_5_both_plus100margin_thickened copy.tif',
 'AML-Raul_M6cn_4_both_plus100margin_thickened copy.tif',
 'AML-Raul_M6cn_7_both_plus100margin_thickened copy.tif',
 'AML-Raul_M6cn_6_both_plus100margin_thickened copy.tif',
 'AML-Raul_M6cn_12_both_plus100margin_thickened copy.tif',
 'AML-Raul_M6cn_13_both_plus100margin_thickened copy.tif',
 'AML-Raul_M6cn_1_both_plus100margin_thickened copy.tif',
 'AML-Raul_M6cn_10_both_plus100margin_thickened co

In [61]:
for file_name in contents:
    if file_name.endswith(("copy.png", "copy.jpg", "copy.jpeg", "copy.tif")):
        try:
            # Detect green lines
            img = cv2.imread(os.path.join(path, file_name))
            if img is None:
                raise ValueError(f"Error loading image: {file_name}")

            hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            
            lower_green = np.array([30, 40, 40])
            upper_green = np.array([90, 255, 255])
            
            mask = cv2.inRange(hsv, lower_green, upper_green)
            
            lines = cv2.HoughLinesP(mask, 1, np.pi/180, threshold=50, minLineLength=10, maxLineGap=10)
            if lines is None:
                raise ValueError(f"No green lines detected in file: {file_name}")

            x1, y1, x2, y2 = lines[0][0]
            
            cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
            
            angle = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi

            # Determine file extensions
            if file_name.endswith("copy.png"):
                b_extension = " copy.png"
                a_extension = ".png"
            elif file_name.endswith("copy.jpg"):
                b_extension = " copy.jpg"
                a_extension = ".jpg"
            elif file_name.endswith("copy.jpeg"):
                b_extension = " copy.jpeg"
                a_extension = ".jpeg"
            elif file_name.endswith("copy.tif"):
                b_extension = " copy.tif"
                a_extension = ".tif"
            
            original_img_path = os.path.join(path, file_name.replace(b_extension, a_extension))
            original_img = cv2.imread(original_img_path)
            if original_img is None:
                raise ValueError(f"Error loading original image: {original_img_path}")

            M = cv2.getRotationMatrix2D((original_img.shape[1]//2, original_img.shape[0]//2), angle, 1)
            
            rotated_img = cv2.warpAffine(original_img, M, (original_img.shape[1], original_img.shape[0]), 
                                         borderMode=cv2.BORDER_CONSTANT, borderValue=(255,255,255))

            cv2.imwrite(original_img_path.replace(a_extension, "_rotated" + a_extension), rotated_img)

        except Exception as e:
            print(f"Error processing file {file_name}: {e}")